In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import multiprocessing
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from multiprocessing import Pool

import pickle

In [7]:
def login():
    with open('credentials.txt', 'r') as f:
        password = f.readline()

    driver = webdriver.Firefox()
    driver.get("https://www.scribophile.com/dashboard/login")
    driver.find_element_by_name('email').send_keys('seokho_hong@yahoo.com')
    driver.find_element_by_name('password').send_keys(password)
    driver.find_element_by_name('password').send_keys(Keys.RETURN)
    return driver
    
def scrape_page(driver, page_num, text_results):
    wait = WebDriverWait(driver, 10)
    wait.until(EC.element_to_be_clickable((By.ID, 'feed')))
    driver.get('https://www.scribophile.com/writing/browse?page=' + str(page_num) + '&genre=fiction&type=novel&showrestrictedwork=false&showlockedwork=false&showpersonalspotlightwork=false&showkarmafreespotlightwork=false&query=&tag=')
    # get the works on the page
    links = []
    for elem in driver.find_elements_by_class_name('work-details'):
        links.append(elem.find_element_by_tag_name('a').get_attribute('href'))
        
    for link in links:
        driver.get(link)
        driver.find_element_by_id('work-body').find_element_by_class_name('pill-button').click()
        text_results[link] = driver.find_element_by_id('work-body').text
    driver.quit()

def run_scraper(page_num, text_results):
    driver = login()
    scrape_page(driver, page_num, text_results)

In [8]:
def scrape_pool(page_range, num_processes=2):
    pool = Pool(num_processes)
    manager = multiprocessing.Manager()
    text_results = manager.dict()
    jobs = []
    for i in page_range:
        pool.apply_async(run_scraper, args=(i, text_results))

    pool.close()
    pool.join()

    return text_results

In [13]:
increment_size = 20
for corpus_index in range(1):
    texts = scrape_pool(range(corpus_index * increment_size, (corpus_index + 1) * increment_size), num_processes=5)
    with open('scribophile/scribophile_texts' + str(corpus_index) + '.txt', 'w') as f:
        f.writelines(texts.values())
    #with open('scribophile_texts' + str(corpus_index) + ".pkl", 'wb') as f:
    #    pickle.dump(texts.items(), f)

In [115]:
with open('scribophile_texts0.pkl', 'rb') as f:
    reloaded_texts = pickle.load(f)